In [ ]:
!pip3 install pycuda
!pip3 install matplotlib

In [ ]:
!mkdir ./figs

In [ ]:
"""
  bz: simulation of the Belousov-Zhabotinsky reaction using a cellular automata 
      approximation with GPU acceleration.  Main reference: A. Turner, 
      "A simple model of the Belousov-Zhabotinsky reaction from first principles"
      Implementation note, UCL, (2009).

    Copyright (C) Cesar L. Pastrana, 2022

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.

"""

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm

import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit


bzmodule =  SourceModule("""
    #include <stdio.h>

   __device__ inline int pbc(int x, int L);
   __device__ inline double moore_mean_conc (double *arr, 
                                             int i, int j, 
                                             int n, int s, int w, int e,
                                             int nc);                                  
  

     /**************************************************************************
     *
     * bz_reaction: Calculates a step in the Belousov-Zhabotinsky reaction 
     *              following a cellular automata approximation with a Mooore 
     *              neighbourhood and periodic boundary conditions. 
     *              The reations considered are:
     *              A+B -> 2A (rate alpha)
     *              B+C -> 2B (rate beta)
     *              C+A -> 2C (rate gamma)
     *    
     *--------------------------------------------------------------------------
     *  Input: Xarr_in = array of X concentrations before reaction
     *         Xarr_in = array of X concentrations after reaction
     *         alpha,beta, gamma = reaction rates for the reactions above
     *         nr = number of rows of the lattice
     *         nc = number of columns of the lattice
     *
     *  Ouput: void
     *
     **************************************************************************/
     __global__ void bz_reaction(double *aarr_in, double *aarr_out,
                                 double *barr_in, double *barr_out,
                                 double *carr_in, double *carr_out,
                                 double alpha, double beta, double gamma,
                                 int nr, int nc)
     {
      
      int i = threadIdx.x + blockIdx.x*blockDim.x;
      int j = threadIdx.y + blockIdx.y*blockDim.y;
      
      // Coordinates in pbc
      int n = pbc(i-1,nr);
      int s = pbc(i+1,nr);
      int w = pbc(j-1,nc);
      int e = pbc(j+1,nc);
     
      // Mean value (Moore's neighbourhood)
      double ma = moore_mean_conc(aarr_in, i, j, n, s, w, e, nc);
      double mb = moore_mean_conc(barr_in, i, j, n, s, w, e, nc);
      double mc = moore_mean_conc(carr_in, i, j, n, s, w, e, nc);
    
      // Reaction
      aarr_out[i*nc+j] =  ma + ma*(alpha*mb - gamma*mc);
      barr_out[i*nc+j] =  mb + mb*(beta*mc -  alpha*ma);
      carr_out[i*nc+j] =  mc + mc*(gamma*ma - beta*mb);

      // Constrain the values to the interval [0,1)
      if(aarr_out[i*nc+j]>1)  aarr_out[i*nc+j] = 1;
      if(aarr_out[i*nc+j]<0)  aarr_out[i*nc+j] = 0;

      if(barr_out[i*nc+j]>1) barr_out[i*nc+j] = 1;
      if(barr_out[i*nc+j]<0) barr_out[i*nc+j] = 0;

      if(carr_out[i*nc+j]>1) carr_out[i*nc+j] = 1;
      if(carr_out[i*nc+j]<0)  carr_out[i*nc+j] = 0;
      
    }


     /**************************************************************************
     *
     * moore_mean_conc: Mean value around position from a 9-points Moore stencil
     *--------------------------------------------------------------------------
     *  Input: arr = input array 
     *         i = central coordiante row
     *         i = central coordiante column
     *         n = north coordinate
     *         s = south coordinate
     *         e = east coordinate
     *         w = west coordinate
     *         nc = total number of colums
     *      
     *  Ouput: m = average value around central point i,j
     *
     **************************************************************************/
    __device__ inline double moore_mean_conc (double *arr, 
                                             int i, int j, 
                                             int n, int s, int w, int e,
                                             int nc)
    {
        double m = arr[n*nc+w]  + // nw
                   arr[n*nc+j]  + // n
                   arr[n*nc+e]  + // ne
                   arr[i*nc+w]  + // w
                   arr[i*nc+j]  + // x
                   arr[i*nc+e]  + // e
                   arr[s*nc+w]  + // sw
                   arr[s*nc+j]  + // s
                   arr[s*nc+e];   // se
        return m/9.0;
    }



     /**************************************************************************
     *
     * pbc: Returns coordinates under peridoic boundary conditions
     *--------------------------------------------------------------------------
     *  Input: x = value to transform
     *         L = lattice size
     *
     *  Ouput: x in pbc 
     *
     **************************************************************************/
    __device__ inline int pbc(int x, int L)
    {
        return  x - (int)floor((double)x/L)*L;
    }

""")



def initilise_array(h, w):
  """
      initialise_array
      Generates random arrays for the concentrations of chemical components A, B, C
      in the interval [0,1]

      Input:    arr = w x h array to initialise
                h = height, number of rows
                w = width, number of columns

      Ouput     a,b,c = Arrays for each chemical species
  """
  a = np.float64(np.random.rand(h,w))
  b = np.float64(np.random.rand(h,w))
  c = np.float64(np.random.rand(h,w))
  return a,b,c



def plot_heat_maps(mtxa, mtxb, mtxc, save=False, saveid=0):
  """
      plot_heat_map
      Plots a heatmap or save it to file
      
      Input:    mtxa = 2D array for chemical A
                mtxb = 2D array for chemical B
                mtxc = 2D array for chemical C
                save = Boolean to indicate if is saved or reppresented
                id = If saved the file is going to be saved with id + name
  """

  #fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,60))
  fig = plt.figure(figsize=(15,60))
  ax1 = fig.add_subplot(131)
  ax2 = fig.add_subplot(132)
  ax3 = fig.add_subplot(133)

  # Remove labels
  ax1.xaxis.set_ticklabels([])
  ax1.yaxis.set_ticklabels([])
  ax2.xaxis.set_ticklabels([])
  ax2.yaxis.set_ticklabels([])
  ax3.xaxis.set_ticklabels([])
  ax3.yaxis.set_ticklabels([])

  # Take out ticks
  ax1.tick_params(axis ='both', which ='both', length = 0)
  ax2.tick_params(axis ='both', which ='both', length = 0)
  ax3.tick_params(axis ='both', which ='both', length = 0)
  
  cm = 'nipy_spectral'
  cm = 'gray'

  ax1.set_title('A', fontsize=20)
  hm1 = ax1.imshow(mtxa, 
                cmap=cm, 
                interpolation ='none', 
                aspect = 'auto',  
                vmin=0.0, vmax=1.0)
  ax1.set_aspect(1.0)

  ax2.set_title('B', fontsize=20)
  hm2 = ax2.imshow(mtxb, 
                cmap=cm, 
                interpolation ='none', 
                aspect = 'auto',  
                vmin=0.0, vmax=1.0)
  ax2.set_aspect(1.0)

  ax3.set_title('C', fontsize=20)
  hm3 = ax3.imshow(mtxc, 
                cmap=cm, 
                interpolation ='none', 
                aspect = 'auto',  
                vmin=0.0, vmax=1.0)
  ax3.set_aspect(1.0)

  # Save or reppresent?
  if save==False:
    plt.ion()
    plt.show()
  else:
    backend_= mpl.get_backend() 
    mpl.use("Agg")  # Prevent showing the plots in Jupyter
    plt.savefig("./figs/" + str(saveid) + "_bz.png", bbox_inches='tight')   
    plt.close()
    mpl.use(backend_) # Reset backend




def bzreact_dynamics(w, h, alpha, beta, gamma, maxtime):
  """
    bzreact_dynamics:

    Input:        h = height, number of rows
                  w = width, number of columns
                  D = Diffusion constant (expressed in units of latice sites)
                  maxtime = Number of iterations
  """

  # We need to explicitely transform to the specific data types because   
  #  CUDA variablesare type-specific
  w = np.int32(w)
  h = np.int32(h)
  alpha = np.float64(alpha)
  beta  = np.float64(beta)
  gamma = np.float64(gamma)

  # Get the function from the CUDA module
  bzreact = bzmodule.get_function("bz_reaction") 

  # Initialisation
  aarr, barr, carr = initilise_array(h,w)
  plot_heat_maps(aarr,barr,carr, False)
  plot_heat_maps(aarr, barr, carr, True, 0)
 # Allocate memory for the arrays in the GPU
  aarr_gpu     = cuda.mem_alloc(aarr.nbytes) 
  aarr_upd_gpu = cuda.mem_alloc(aarr.nbytes)
  barr_gpu     = cuda.mem_alloc(barr.nbytes) 
  barr_upd_gpu = cuda.mem_alloc(barr.nbytes)  
  carr_gpu     = cuda.mem_alloc(carr.nbytes) 
  carr_upd_gpu = cuda.mem_alloc(carr.nbytes) 
  

  # Iterate the function on the GPU 
  # We first define the minimal unit ie, the block size. A grid is then a 
  # a collection of blocks, so we attribute a number of blocks that divisible  
  # with the block size considering the size of the array. 
  bw=32   
  bh=32
  gw=int(np.floor(w/bw))
  gh=int(np.floor(h/bh))

  for t in tqdm(range(0, maxtime)):
    # Send data from computer to GPU 
    cuda.memcpy_htod(aarr_gpu, aarr)
    cuda.memcpy_htod(barr_gpu, barr)
    cuda.memcpy_htod(carr_gpu, carr)
  
    # Reaction
    bzreact(aarr_gpu, aarr_upd_gpu,
            barr_gpu, barr_upd_gpu,
            carr_gpu, carr_upd_gpu,
            alpha, beta, gamma,
            w, h,
            block=(bw,bh,1), grid=(gw,gh))
    
    # Retrieve the data from the GPU to the computer
    cuda.memcpy_dtoh(aarr, aarr_upd_gpu)
    cuda.memcpy_dtoh(barr, barr_upd_gpu)
    cuda.memcpy_dtoh(carr, carr_upd_gpu)
    #cuda.Context.synchronize()

    plot_heat_maps(aarr, barr, carr, True, t+1)


  plot_heat_maps(aarr, barr, carr, False)

  return aarr,barr,carr



if __name__ == "__main__":
    w = 512
    h = 512
    alpha = 1.2
    beta = 1.0
    gamma = 1.0
    maxtime = 300
    Null, Null, Null = bzreact_dynamics(w, h, alpha, beta, gamma, maxtime)

In [ ]:
!zip -r -q figs.zip ./figs
!rm -r ./figs